In [3]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.colors as mcolors
#%matplotlib qt


df = pd.read_csv("./data/Electric_Vehicle_Population_Data_20231017.csv")

print(df.columns)

mcl_blue = mcolors.TABLEAU_COLORS['tab:orange']
mcl_orange = mcolors.TABLEAU_COLORS['tab:blue']


Index(['VIN (1-10)', 'County', 'City', 'State', 'Postal Code', 'Model Year',
       'Make', 'Model', 'Electric Vehicle Type',
       'Clean Alternative Fuel Vehicle (CAFV) Eligibility', 'Electric Range',
       'Base MSRP', 'Legislative District', 'DOL Vehicle ID',
       'Vehicle Location', 'Electric Utility', '2020 Census Tract'],
      dtype='object')


In [ ]:
%%html
<style>
body {
    font-family: "Century gothic";
}
</style>    

#import plotly.io as pio
#pio.kaleido.scope.default_format = "png"
import plotly.io as pio
pio.renderers.default = 'notebook'


<h2>Introduction</h2>
<p>The invetion of electricity had a great impact on the society. It opened many posibilities to the new innovations including electric cars. Earliest forms of electric vehicle appeared in early 1800s but the functional ones made it to the road around 1890s. Since electric vehicles were invented, although their popularity had ebb and flow they lasted until today. As the times changed, there is a growing demand for electric vehicles today.</p>
<p>In this jupyter notebook, we'll go through tha data of Battery Electric Vehicles (BEVs) and Plug-in Hybrid Electric Vehicles (PHEVs) that are currently registered through Washington State Department of Licensing (DOL).</p>
<p>Database 'Electric Vehicle Population Data' is provided by Washington State Department of Licensing which is under the ODbL.</p>

<h2>Rough insight</h2>
<p>Firstly, to get the rough insight in data, we'll plot a graph of the Model Year of registrated cars.</br>
As we can see, there are much more registered models manufactured in recent years than before. This can be attributed to the lifetime of the cars, as the significantly older cars are more likely to malfunction. Another reason can be the growing awareness of global warming, which was reflected on the growth of offered electric vehicle models and demand from the citezens for the cleaner energy solutions.</p>

In [ ]:
# Registered models by years

fig = px.histogram(df,x="Model Year")
fig.update_layout(bargap=0.2, title={"text": "Number of vehicle models from certaint year"}, yaxis_title="Number of vehicles", 
                  xaxis = dict(tickmode='linear', tick0 =df['Model Year'].min(), dtick = 1.0)
                  )
fig.update_traces( texttemplate="%{y:d}")
fig.show(renderer='notebook')
print('Figure 1. Number of vehicle models from certaint year')


<h2>Vehicle Types</h2>
<p>Electric vehicles are divided in two categories, Battery Electric Vehicle (BEV) and Plug-in Hybrid Electric Vehicle (PHEV). Both of those types can be charged by the plug-in, but there is one importatnt difference. BEVs are driven only by the electric battery, while the PHEV also contain a fuel engine combined with the electric one. That difference is significant when analysing the Electric Range of the vehicle in later cells, because it considers only the strength of the electric engine, ignnoring the fuel driven parts of PHEV.</p>

<h3>Annual change</h3>
<p>Next we look separately at a numebr of BEVs and PHEVs by the Model Year, and this distribution can be seen in figure 2. The growth of both types began around 2010, where the exponential grow begins, which is also the time period when the PHEVs were introduced. In the 2020s, we can see the different trends of the Electric Vehicle Types, where PHEVs have less growth of registered vehicle models than BEVs over years. This can be connected to the growing awereness of the global warming, as the BEVs have cleaner energy source than the PHEVs which combines electric energy with other fuels </p>


In [ ]:
# Electric Vehicle Types through years
tmpdf = df[df['Model Year']>=2010]
fig = go.Figure(data=[go.Histogram(x=tmpdf[tmpdf['Electric Vehicle Type']=='Plug-in Hybrid Electric Vehicle (PHEV)']['Model Year'],
                                   name='Plug-in Hybrid Electric Vehicle (PHEV)', marker=dict(color=mcl_blue), texttemplate="%{y:d}"),
                      go.Histogram(x=tmpdf[tmpdf['Electric Vehicle Type']=='Battery Electric Vehicle (BEV)']['Model Year'],
                                    name='Battery Electric Vehicle (BEV)', marker=dict(color=mcl_orange),  texttemplate="%{y:d}" ), 
                      ]
                )
#fig.update_yaxes(type="log")
fig.update_layout(legend=dict(yanchor="bottom",y=1.02, xanchor="right",x=1))
fig.update_layout(bargap=0.2, title={"text": "Number of BEVs and PHEVs from 2010"}, yaxis_title="Number of vehicles", xaxis_title="Model Year", 
                  xaxis = dict(tickmode='linear', tick0 =df['Model Year'].min(), dtick = 1.0)
                  )

fig.show(renderer='notebook')

print('Figure 2. Number of vehicle models from certaint year of BEVs and PHEVs')


<h2>Battery Range</h2>
<h3>Electric Vehicle Type and the Battery Range</h3>
<p>When we are talking about Electric Vehicle Types, we can additionally divide them by category of Clean Alternative Fuel Vehicle (CAFV) Eligibility, which is closely conneted to the Battery Range and the Electirc Vehicle Type. But except the labels 'Clean Alternative Fuel Vehicle Eligible' and 'Not eligible due to low battery range' there is a significant number of vehicles with the 'Eligibility unknown as battery range has not been researched'. This also appears in the Battery Range as the value 0, which is not an expected value to have. The cause for this is the fact that the Electric Ragne is no longer maintained for the BEVs. In the later discussions, we will sometimes exclude those informations to maintain quality of data.</p>

In [ ]:
# Battery types and pie chart
#print(set(df['Clean Alternative Fuel Vehicle (CAFV) Eligibility']))

tmpdf = df.value_counts(['Electric Vehicle Type','Clean Alternative Fuel Vehicle (CAFV) Eligibility']).to_frame('makens').reset_index().sort_values('Electric Vehicle Type')
print('Table 1. Numbers of Electric Vehicle Types Clean Alternative Fuel Vehicle (CAFV) Eligibility ')
display(tmpdf)
print('\n')

fig, ax = plt.subplots()
size = 0.4

typs = ['Battery Electric Vehicle (BEV)', 'Plug-in Hybrid Electric Vehicle (PHEV)']
tmpdat = [tmpdf['makens'][tmpdf['Electric Vehicle Type']==i].to_numpy() for i in typs]



cmap = plt.colormaps["tab20c"]
inner_colors = cmap([0, 4])
outer_colors = cmap([1, 2, 3, 5, 6])

ax2 = ax.twinx()
ax.pie([sum(i) for i in tmpdat], radius=1, colors=inner_colors,
       wedgeprops=dict(width=size, edgecolor='w'), 
       labels=['Battery Electric\nVehicle (BEV)','Plug-in Hybrid\nElectric Vehicle (PHEV)'],
       labeldistance=1.1, autopct='%1.2f%%', pctdistance=0.8, textprops=dict(color="k", size=12))

#ax.legend([1,2], loc='upper right')

ax2.pie([i for subset in tmpdat for i in subset], radius=1-size, colors=outer_colors,
       wedgeprops=dict(width=size, edgecolor='w'), autopct='%1.2f%%', pctdistance=0.67, textprops=dict(size=9))

ax2.legend(tmpdf['Clean Alternative Fuel Vehicle (CAFV) Eligibility'],  loc='center left',bbox_to_anchor=(0.8, 1.))
#ax2.set(aspect="equal", title='nesto')

plt.show()

print("Figure 3. Ratio of Electric Vehicle Types and percentages of individual Clean Alternative Fuel Vehicle (CAFV) Eligibility category")

<h3>Exceptions</h3>
<p>To know what is sometimes being excluded, we provide a small table with information of vehicles with the lable 'Eligibility unknown as battery range has not been researched'. Looking in table 2, we can notice that those data are mainly from 2020s, likely around the time when the 'Electric Range' category was no longer maintained because new BEVs have an electric range of 30 miles or more.</p>
<p>Additionally there was a small amount of BEVs with label 'Not eligible due to low battery range'. When we unwrapped the data like in table 3, there was only one type of vehicle, which is Hyundai Ioniq 2019. After checking the vehicles of same Make, Model Year and Model, the same models of BEV and PHEV were listed. Intrestingly, the 'weird' data is the mix of BEV's label and PHEV's Electric range.</p>

In [ ]:
# Without data about battery
print("Table 2. Number of vehicles with label 'Eligibility unknown as battery range has not been researched'")
tmpdf = df[df['Clean Alternative Fuel Vehicle (CAFV) Eligibility']=='Eligibility unknown as battery range has not been researched']
tmpdf = tmpdf.value_counts(['Model Year','Electric Vehicle Type']).to_frame('batNs').reset_index()

display(tmpdf)

# Not eligible due to low battery range, BEV
print('\n'*2+"Table 3.a. All BEV vehicles with lable 'Not eligible due to low battery range':")
tmpdf = df[df['Clean Alternative Fuel Vehicle (CAFV) Eligibility']=='Not eligible due to low battery range']
tmpdf = tmpdf[tmpdf['Electric Vehicle Type']=='Battery Electric Vehicle (BEV)']

tmpdf = tmpdf.loc[:,['Model Year','Make','Model','Electric Range']].drop_duplicates()
display(tmpdf)

# in all data
print('\n'+'Table 3.b. Hyundai Ioniq\'s Electric Range')
tmpdf = df.loc[:,['Model Year','Make','Model','Electric Range','Electric Vehicle Type']].drop_duplicates()
tmpdf = tmpdf[np.logical_and(tmpdf['Make']=='HYUNDAI', tmpdf['Model']=='IONIQ')]
tmpdf = tmpdf[tmpdf['Model Year']==2019]
display(tmpdf.drop_duplicates())

<h3>'Two-way models'</h3>
<p>As was the case with Hyundai Ioniq, we'll check for another 'two-way' vehicles wich come in both BEV and PHEV models. After filtering the data, the results are shown in the table 4. It appears that only the three vehicle models came in the both versions.</p>

In [ ]:
# only by models
tmpdf = df.loc[:,['Make','Model','Electric Vehicle Type']].drop_duplicates()
tmpdf = tmpdf.value_counts(['Make','Model']).to_frame('modNs').reset_index()
print('Out of %d car models, there are only %d two way models' % (len(tmpdf), sum(tmpdf['modNs']==2)))

print("\nTable 4. List of 'two way' vehicles ")
display(tmpdf[tmpdf['modNs']>1])

<h3>Battery Range by Electric Vehicle Type</h3>
<p>Let's get back to the Battery Range. First, as the Battery Range has a different significance to the BEVs and PHEVs, we are separately analysing the Battery Ranges. The difference is prominent in figure 4 and is separated with the different colours, where the PHEVs have much less Battery range than BEVs, which is reasonable as PHEVs has additional energy source.</p>
<p>In figure 4, we present the number of vehicles with the same Electric Range as areas of circles around the Electric Range point. It simplified the picture from the quite repeating Electric range values, as there are many vehicles of same models or similar engines.</p>
<p>While looking at the distributed data, we can see a significant gap in BEVs Electric Range, which is around 175. Also, above that value, there are few models with notable vehicle counts. With that reason in mind, we plot an another graph 4.b, which labels those vehicles with their Make. As it appears, the 'TESLA' vehicles commonly had the biggest Battery Ranges. </p>



In [ ]:
# Excluding cars without electric range
from matplotlib.lines import Line2D
tmpdf = df[df['Electric Range'] !=0 ]
print('Different values of Electric Range: %d'%len(tmpdf.value_counts('Electric Range')))
print('Number of vehicles with known Electric range: %d'%len(tmpdf))

fig, ax = plt.subplots(2,1,figsize=(10,10), gridspec_kw={'height_ratios': [5, 3]}, sharex=True)
hlloc = 175

# counting cars
tmpdf = tmpdf.value_counts(['Model Year','Electric Range','Electric Vehicle Type']).to_frame('carNs').reset_index()
tmpdf = tmpdf[tmpdf['Model Year']>=2010]

carTypes = tmpdf['Electric Vehicle Type'].drop_duplicates().to_list()
print('Vehicle types are: %s' % carTypes)

print('\n'*2+'Figure 4. Electric Range of vehicles by Model Year')

# plot firts graph, separately BEVs and PHEVs
for i in carTypes:
    typdf = tmpdf[tmpdf['Electric Vehicle Type']==i].sort_values('Model Year')

    tmpx = np.array([])

    for j in typdf['Model Year'].drop_duplicates():
        tmpx = np.append(tmpx, np.linspace(j-0.5,j+0.5,sum(typdf['Model Year']==j)+2)[1:-1])
    
    ax[0].scatter(typdf['Model Year'].to_numpy(), typdf['Electric Range'].to_numpy(), s=typdf['carNs'].to_numpy()*.3, alpha=0.6,label=i)    
    ax[0].scatter(typdf['Model Year'].to_numpy(), typdf['Electric Range'].to_numpy(), marker='D', s=2, color='#0f0f0f',label='_nolegend_')

ax[0].hlines(hlloc,2010-0.5,2024+0.5, 'gray', linestyle=':')

ax[0].set_ylabel('Electric Range')
ax[0].set_xticks(np.arange(2010,2025))
ax[0].xaxis.set_tick_params(which='both', labelbottom=True)
ax[0].grid(which='major', axis='x', color='#808080', alpha=0.4, linestyle=':')
ax[0].legend(carTypes,markerscale=0.5)
ax[0].text(-0.05, 1.05, 'a)', horizontalalignment='left', verticalalignment='top',transform=ax[0].transAxes)

# plot second graph, BEVs with bigger Electric Range
tmpdf = df[df['Electric Range'] !=0 ]
tmpdf = tmpdf[tmpdf['Model Year']>=2010]
tmpdf = tmpdf.loc[:,['Model Year','Make','Electric Range','Electric Vehicle Type']]
tmpdf = tmpdf[tmpdf['Electric Range']>180].sort_values('Model Year').drop_duplicates()

cmap = matplotlib.colormaps['tab10']
cldct = {list(set(tmpdf['Make']))[i]:cmap(i/len(set(tmpdf['Make']))) for i, ii in enumerate(set(tmpdf['Make']))}

scc=ax[1].scatter(tmpdf['Model Year'].to_numpy(), tmpdf['Electric Range'].to_numpy(), 
                  c=[cldct[i] for i in tmpdf['Make'].to_numpy()], edgecolor='#080808', s=6**2, alpha=0.7)

legleg = [Line2D([0], [0], marker='o', markeredgecolor='#080808',markerfacecolor=cldct[i], color='w', label=i, markersize=6) for i in cldct.keys()]
ax[1].legend(handles=legleg)

ax[1].set_yticks([200,250,300,350])
ax[1].set_ylabel('Electric Range')
ax[1].xaxis.set_tick_params(which='both', labelbottom=True)
ax[1].grid(which='major', axis='x', color='#808080', alpha=0.4, linestyle=':')
ax[1].text(-0.05, 1.1, 'b)', horizontalalignment='left', verticalalignment='top',transform=ax[1].transAxes)


plt.xlabel('Model year')
plt.xlim([2010-0.5,2024+0.5])
plt.show()



<h2>Make</h2>
<p>From figure 5, we can spot the differences between Makes and their share. The make 'TESLA' has the largest share in electric vehicles with only registered BEVs. Second place is 'NISSAN' which also has only registered BEVs. On the other hand, there are Makes only with PHEVs like 'JEEP' or 'CHRYSLER'. Detalied preview of the Makes is shown in table 5, with the Makes separated by registered Vehicle Types.</p>


In [ ]:
# Makers and the battery type
fig3 = px.histogram(df, x="Make", color='Electric Vehicle Type', color_discrete_map = {'Battery Electric Vehicle (BEV)':mcl_orange, 'Plug-in Hybrid Electric Vehicle (PHEV)':mcl_blue}, log_y=True)
fig3.update_layout(xaxis={'categoryorder':'total descending'})
fig3.update_layout(bargap=0.2, yaxis_title="Number of vehicles", xaxis_title="Make")
fig3.update_layout(legend=dict(yanchor="bottom",y=1.02, xanchor="right",x=1))


fig3.show(renderer='notebook')
print('Figure 5. Number of vehicles by Make')

# table plot
tmpdf = df.value_counts(['Make','Electric Vehicle Type']).to_frame('Ns').reset_index()

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'table'},{'type':'table'},{'type':'table'}]])

sbev = set(tmpdf[tmpdf['Electric Vehicle Type']=='Battery Electric Vehicle (BEV)']['Make'])
sphev = set(tmpdf[tmpdf['Electric Vehicle Type']=='Plug-in Hybrid Electric Vehicle (PHEV)']['Make'])

fig.add_trace(go.Table(header=dict(values=['BEV']), cells=dict(values=[np.sort(list(sbev-sphev))] )), row=1, col=1)
fig.add_trace(go.Table(header=dict(values=['BEV & PHEV']), cells=dict(values=[np.sort(list(sbev & sphev))] )), row=1, col=2)
fig.add_trace(go.Table(header=dict(values=['PHEV']), cells=dict(values=[np.sort(list(sphev-sbev))] )), row=1, col=3)

fig.update_layout(height=600)

print('\nTable 5. Makes by the Electric Vehicle Type')
fig.show(renderer='notebook')


<h3>Car Models through years</h3>
<p>Most companies have multiple vehicle models on the market, which is mirrored in our dataset. It is common for vehicle models to be produced throughout years with improved specifications. In figure 6, the vehicle Models are listed by the Makes and the registered Model Years. It is intresting that the number of various models does not directly indicate a lager share of vehicles. For the example, NISSAN on the second place by the registred number of vehicles has only two Models.</p>

In [ ]:
%matplotlib inline
tmpdf = df.value_counts(['Make','Model','Model Year']).to_frame('makeNs').reset_index().sort_values('Model Year')


fig, ax = plt.subplots(figsize=(7,50))
mklocs = {}
n_makes = len(set(tmpdf['Make']))

# number of sold cars of one maker
lili = df.value_counts(['Make']).to_frame('Ns').reset_index().sort_values('Ns')


# for every Make
for i, inm in enumerate(lili['Make']):
    tmpx = []

    # little marks of PHEV and BEV
    if inm in sbev:
        plt.scatter([tmpdf['Model Year'].min()-1], [i+0.25], c=mcl_blue)
    if inm in sphev:
        plt.scatter([tmpdf['Model Year'].min()-1], [i-0.25], c=mcl_orange)
    #print(inm)
    mkdf = tmpdf[tmpdf['Make']==inm]
    mklocs['inm'] = i

    tmpx = np.append(tmpx, np.linspace(i-0.5, i+0.5, len(mkdf.drop_duplicates('Model'))+2 )[1:-1] )

    tmpmodels = mkdf['Model'].drop_duplicates().to_list()
    #print(tmpmodels)
    for j, jnm in enumerate(tmpmodels):
        jmkdf = mkdf[mkdf['Model']==jnm]
        #print(jmkdf)

        plt.plot(jmkdf['Model Year'].to_numpy(), np.ones(len(jmkdf['Model Year'].to_numpy()))*tmpx[j], 
                 marker='|', markersize=10, 
                 color=matplotlib.colors.hsv_to_rgb([i/n_makes, 0.8, 0.5+j/len(tmpx)*.5])) 

        # special cases when the lable is too long
        if not jnm in ['RANGER','S-10 PICKUP']:
            ax.annotate(jnm+' ', xy=(jmkdf['Model Year'].min(), tmpx[j]), ha='right', va='center')
        else:
            ax.annotate(jnm+' ', xy=(jmkdf['Model Year'].min(), tmpx[j]+0.02), ha='left', va='bottom')
        #mkdf[mkdf[]

    #print(tmpx)
    #print(mkdf)

plt.yticks(np.arange(i+1),[i for i in lili['Make']], rotation=90, va='center')
plt.xticks(np.arange( tmpdf['Model Year'].min(), tmpdf['Model Year'].max()+1 ), rotation=90)
plt.yticks(np.arange(-0.5, i+1+0.5, 1), minor=True)
ax.tick_params(axis='y', which='minor', left=False)
plt.xlim([tmpdf['Model Year'].min()-1, tmpdf['Model Year'].max()+1])
plt.ylim([-0.5,i+0.5])

plt.xlabel('Year')
plt.ylabel('Make')


plt.grid(which='minor')
plt.rc('grid', linestyle="--", color='gray')

fig.tight_layout()

plt.show()
print('Figure 6. Vehicle Models throug Model Years gruouped by Make')



<h2>Geographical distribution</h2>
<p>Besides the numerical and categorical data, it's practical to visualise data linked to the geographical locations. In the dataset, there are informations of vehicles location going from the State, County and point location defined by latitude and longitude. First, we'll plot the vehicle location information by the States in figure 7 and 8. It's not suprising thet the most registered electric vehicles in the Washington state are located there, but the unexpected number of vehicles are located in the California state with 92 vehicles. Also states VA and MD have a significant number of registered vehicles, but the more intresting fact is their location, because those states are on the oposite coastline of the USA.</p>

In [ ]:
tmpdf = df.value_counts('State').to_frame('stateNs').reset_index()

fig = px.bar(tmpdf, x='State',y='stateNs', log_y=True, labels={'stateNs':'Vehicle number in a state'})
fig.update_traces( texttemplate="%{y:d}")
fig.show(renderer='notebook')
print('Figure 7. Vehicle numbers by the States')

In [ ]:
import plotly.express as px

fig = px.choropleth(locations=tmpdf['State'], locationmode="USA-states", color=tmpdf['stateNs'], scope="usa",
                    range_color=(0,100))
fig.update_layout(coloraxis=dict(colorbar=dict(title_text='Vehicle number',title_side='right')))
fig.show(renderer='notebook')
print('Figure 8. Vehicle numbers by the States on the map')

<h3>Washington state</h3>
<p>As the Washington state has the largest number of electric vehicles registered, we are closely looking at the state. While we are presenting the number of vehicles in certaint points, we'll additionally show the rate of BEVs to PHEVs which is marked by the color scale. </br>
In figure 9, we can see the concentrations of registered vehicles to towns. Intresting thing to look for is the overall rate of Electric Types by the bigger cities. Around Seattle, mainly BEVs are dominant, while in other cities BEVs are also dominant, but with a smaller rate. </p>

In [ ]:
tmpdf = df[df['State']=='WA']
tmpdf = tmpdf.value_counts(['Electric Vehicle Type', 'Vehicle Location']).to_frame('pNs').reset_index()
%matplotlib inline

# data
xs = []
ys = []
ss = []
cs = ["#ff0000" if i=="Battery Electric Vehicle (BEV)" else "#0000ff" for i in tmpdf['Electric Vehicle Type']]

for i in tmpdf['Vehicle Location']:
    tmp = i.split('(')[1].split(')')[0].split(' ')
    xs.append(float(tmp[0]))
    ys.append(float(tmp[1]))

tmpdf['lon'] = xs
tmpdf['lat'] = ys


def func(group):
    #print(group)
    if len(group)==1:
        #print(group['Electric Vehicle Type'] )
        #print('Battery Electric Vehicle (BEV)' in group['Electric Vehicle Type'].to_list())
        #print('\n')
        return pd.Series({'e_pc':1, 'lbl':group['pNs'].sum()}) if "Battery Electric Vehicle (BEV)" in group['Electric Vehicle Type'].to_list() else pd.Series({'e_pc':-1, 'lbl':group['pNs'].sum()})
    else:
        return pd.Series({'e_pc': (1-2*(group[group['Electric Vehicle Type']=='Plug-in Hybrid Electric Vehicle (PHEV)']['pNs'].to_numpy()[0] / group['pNs'].sum())), 'lbl':group['pNs'].sum()})

grouped = tmpdf.groupby(['Vehicle Location','lon','lat']).apply(func)
grdf = grouped.reset_index()

# interactive map
fig = go.Figure(go.Scattermapbox(
        lat=grdf.lat,
        lon=grdf.lon,
        mode='markers',
        marker_colorscale="jet_r",
        marker=dict(
            size=grdf['lbl']**0.5*0.5,
            color=grdf['e_pc'],              
            opacity=0.8,
            colorbar=dict(thickness=20, title_text='--nesto--',title_side='right')
        ),
        text=grdf['lbl'],
        hovertemplate='Point(%{lon},%{lat})<br>Vehicle count:%{text}'
    ),layout=go.Layout(
        mapbox_style="open-street-map",
        mapbox_bounds={"west": grdf.lon.min()-2, "east": grdf.lon.max()+2, "south":grdf.lat.min()-2, "north": grdf.lat.max()+2},
        margin={'r':10,'l':10,'b':10,'t':10},       
        ))

fig.show(renderer='notebook')

print('Figure 9. Locations of electric vehicles and the ratio BEV to PHEV in the point')



<h3>Exceptions</h3>
<p>On the map, we can see two bigger points of deviation from the sorounding data in King County. At the same time, those two point has largest number of vehicles while the ratios BEV to PHEV are less than 0, as shown in the table 6. That indicates a bigger share of PHEVs in those points, which are sorounded by opposit ratios. When we look closer to the location on the map, it is located quite near the Seattle-Takoma International Airport and the Renton Municipial Airport next to the Boeing Renton Plant.</p>
<p>If we single out some Makes based on their share in those points like presented in figure 10 and 11, we can see that the dominant PHEV Makes are 'JEEP' and 'CHRYSLER' in both points. It can be connected to the different demands of the airport from the usual urban neighborhoods.</p>

In [ ]:
print('Table 6. Points with more PHEVs then BEVs')
display(grdf[grdf['e_pc']<0].sort_values('lbl', ascending=False).head(10))

cnt = 10
lblcnt = 0
lbl = ['Seattle-Tacoma Inetrnational Airport', 'Renton Municipial Airport']
for i in ['POINT (-122.29179 47.43473)', 'POINT (-122.21024 47.4797047)']:
    tmpdf = df[df['Vehicle Location']==i].sort_values('Make')
    
    fig = px.histogram(tmpdf, x='Make', color='Electric Vehicle Type',color_discrete_map = {'Battery Electric Vehicle (BEV)':mcl_orange, 'Plug-in Hybrid Electric Vehicle (PHEV)':mcl_blue})
    fig.update_layout(legend=dict(yanchor="bottom",y=1.02, xanchor="right",x=1))
    fig.show(renderer='notebook')
    print('Figure %d. Number of vehicles by the Make in the %s near the %s' % (cnt, i, lbl[lblcnt]))
    
    cnt+=1
    lblcnt+=1

